In [ ]:
import urllib.request
import json
import os
import ssl
from PIL import Image
from io import BytesIO
import base64
import matplotlib.pyplot as plt

In [ ]:
data = {"prompt": "A fantasy style highly detailed drawing of a knight fighting a dragon on the style of Greg Rutkowski",
       "image":""}  

data1 = {"prompt": "A photorealistic highly detailed painting of a sorceress wearing green robes on the style of Gustav Klimt",
       "image":""}  

data2 = {"prompt": "A portrait of UK Prime Minister Boris Johson wearing a dark suit, Grand Theft Auto Cover art style, detailed and colorful",
       "image":""} 

data3 = {"prompt": "Grand Theft Auto style painting of a bearded man wearing glasses and a baseball hat, sharp lines",
       "image":""} 

data4 = {"prompt": "A detailed painting of the cloudy Seattle Skyline, Trending On deviant art colorful",
       "image":""} 

data5 = {"prompt": "A detailed painting of the Elmo raising his arms melting in front of a fiery pit, Greg Rutkowski",
       "image":""} 

       #"contentType": "JPEG"

In [ ]:

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

def call_img2img_endpoint(data):

    body = str.encode(json.dumps(data))

    #You can find these in the consume section of your endpoint
    url = 'YOUR_URL_HERE'
    api_key = 'YOUR_KEY_HERE' # Replace this with the API key for the web service

    # The azureml-model-deployment header will force the request to go to a specific deployment.
    # Remove this header to have the request observe the endpoint traffic rules
    headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key), 'azureml-model-deployment': 'default' }

    req = urllib.request.Request(url, body, headers)

    try:
        response = urllib.request.urlopen(req)

        result = response.read()
        print(result)
    except urllib.error.HTTPError as error:
        print("The request failed with status code: " + str(error.code))

        # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
        print(error.info())
        print(error.read().decode("utf8", 'ignore'))

    return result


In [ ]:
file = os.path.join("images", "elmo.jpg")
img = Image.open(file)

buffered_result_image = BytesIO()

img.save(buffered_result_image, format="PNG")
result_image_str = base64.b64encode(buffered_result_image.getvalue()).decode('utf-8')
data5["image"] = result_image_str
data5["strength"] = .85

img

In [ ]:
result = call_img2img_endpoint(data5)
result_dict = json.loads(result.decode('utf-8'))

In [ ]:
def display_result_images(result_dict):

    if "result" in result_dict:
        # if it's only one image
        im = Image.open(BytesIO(base64.b64decode(result_dict["result"])))
        plt.imshow(im)
    else :
        plt.figure(figsize=(20,30))
        columns = 3
        for i, img_b64 in enumerate(result_dict["results"]):
            plt.subplot(3, columns, i + 1)
            image = Image.open(BytesIO(base64.b64decode(img_b64)))
            plt.imshow(image)

In [ ]:
display_result_images(result_dict)